# Annotating Ground Truth

In this notebook, we'll walk through a simple example of how you can annotate ground truth data for temporal events using VGrid. We'll look for drug commercials in TV News videos.

## Load Data

We'll start by loading up the TV news videos and commercial breaks.

In [1]:
from vgrid import VGridSpec, VideoMetadata, VideoBlockFormat, SpatialType_Bbox, SpatialType_Caption
from vgrid_jupyter import VGridWidget
from rekall import Interval, IntervalSet, IntervalSetMapping, Bounds3D
from rekall.stdlib import ingest
from rekall.predicates import *
import urllib3, requests, os
urllib3.disable_warnings()

VIDEO_COLLECTION_BASEURL = "http://olimar.stanford.edu/hdd/rekall_tutorials/workshop"
VIDEO_METADATA_FILENAME = "data/video_meta.json"
VIDEO_ENDPOINT = "http://olimar.stanford.edu/hdd/rekall_tutorials/workshop/videos"

req = requests.get(os.path.join(VIDEO_COLLECTION_BASEURL, VIDEO_METADATA_FILENAME), verify=False)
video_collection = req.json()

video_metadata = [
    VideoMetadata(v["path"], v["id"], v["fps"], int(v["num_frames"]), v["width"], v["height"])
    for v in video_collection
]

# Load commercial annotations
COMMERCIAL_JSON = "data/commercials.json"
req = requests.get(os.path.join(VIDEO_COLLECTION_BASEURL, COMMERCIAL_JSON), verify=False)
commercials_json = req.json()

commercials_ism = ingest.ism_from_iterable_with_schema_bounds3D(
    commercials_json,
    ingest.getter_accessor,
    {
        'key': 'video_id',
        't1': 'start',
        't2': 'end'
    },
    with_payload = lambda item: item,
    progress = True
)

def generate_spec(isms):
    return VGridSpec(
        video_meta = video_metadata,
        vis_format = VideoBlockFormat(imaps = [
            (str(i), ism)
            for i, ism in enumerate(isms)
        ]),
        video_endpoint = VIDEO_ENDPOINT
    ).to_json_compressed()

100%|██████████| 49/49 [00:00<00:00, 180916.28it/s]


## Visualize the data

Next, we'll visualize the data. But notice that we don't use the exact same code as we've been doing so far - instead, we'll get a pointer to the VGridWidget first before displaying it.

Go ahead and run the next cell, and then come back to the annotation instructions below.

### Annotating Data
Once you've run the next cell, you should see the Vgrid interface below. Click on one of the videos to expand it, and start looking for a drug commercial.

**Once you've found a drug commercial, hover over the timeline and press `i` to indicate the beginning of a segment. Play the video until the end of the segment (or just click through until you find the end of the segment), and press `i` again to indicate the end of that segment. You should see your annotation appear on the timeline.**

### Saving Annotations

Once you've found a few annotations, go ahead to the next section to learn how to save them.

In [4]:
widget = VGridWidget(vgrid_spec = generate_spec([commercials_ism]))
widget

VGridWidget(vgrid_spec={'compressed': True, 'data': b'x\x9c\xcd\x9a\xdfO\xe38\x10\xc7\xff\x95U\x9fO\xd4\x1e\xf…

## Saving Annotations

When you make annotations, they're reflected in of `widget.label_state` (until you re-run the cell or refresh the page). Take a look at it below:

In [5]:
widget.label_state

{'blocks_selected': {},
 'block_labels': {'0': {'captions_selected': [],
   'new_intervals': [{'bounds': {'t1': 1931.73003,
      't2': 2016.2642103586563,
      'bbox': {'x1': 0, 'x2': 1, 'y1': 0, 'y2': 1}},
     'data': {'spatial_type': {'args': {}}, 'metadata': {}}}]},
  '1': {'captions_selected': [], 'new_intervals': []},
  '2': {'captions_selected': [], 'new_intervals': []},
  '3': {'captions_selected': [], 'new_intervals': []},
  '4': {'captions_selected': [], 'new_intervals': []},
  '5': {'captions_selected': [], 'new_intervals': []},
  '6': {'captions_selected': [], 'new_intervals': []},
  '7': {'captions_selected': [], 'new_intervals': []},
  '8': {'captions_selected': [], 'new_intervals': []}}}

In [6]:
widget.label_state['block_labels']

{'0': {'captions_selected': [],
  'new_intervals': [{'bounds': {'t1': 1931.73003,
     't2': 2021.721341,
     'bbox': {'x1': 0, 'x2': 1, 'y1': 0, 'y2': 1}},
    'data': {'spatial_type': {'args': {}}, 'metadata': {}}}]},
 '1': {'captions_selected': [], 'new_intervals': []},
 '2': {'captions_selected': [], 'new_intervals': []},
 '3': {'captions_selected': [], 'new_intervals': []},
 '4': {'captions_selected': [], 'new_intervals': []},
 '5': {'captions_selected': [], 'new_intervals': []},
 '6': {'captions_selected': [], 'new_intervals': []},
 '7': {'captions_selected': [], 'new_intervals': []},
 '8': {'captions_selected': [], 'new_intervals': []}}

`widget.label_state['block_labels']` stores new intervals for each Vgrid block. You'll need to re-associate with block with its video ID, and then store the results somewhere. Here's an example of doing just that:

In [11]:
# Get the video IDs from commercials_ism. Vgrid always displays videos sorted by the key.
video_ids = sorted(list(commercials_ism.keys()))

# Store the temporal extents of new Intervals.
drug_commercial_annotations = {}
for block_num, video_id in zip(widget.label_state['block_labels'], video_ids):
    block_labels = widget.label_state['block_labels'][block_num]
    
    drug_commercial_annotations[video_id] = [
        { 't1': interval['bounds']['t1'], 't2': interval['bounds']['t2'] }
        for interval in block_labels['new_intervals']
    ]
drug_commercial_annotations

{17458: [{'t1': 1931.73003, 't2': 2021.721341}],
 19882: [],
 34359: [],
 34642: [],
 37170: [],
 38275: [],
 42756: [],
 52945: [],
 55711: []}

In [12]:
import json

# Save them to disk
with open('drug_commercial_annotations.json', 'w') as f:
    json.dump(drug_commercial_annotations, f)